LIMPEZA DOS DADOS

In [ ]:
import sklearn as sk
import matplotlib as mt
import seaborn as sb
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [ ]:
df = pd.read_csv("dataset.csv")

In [ ]:
# A base ta limpinha
print(df)
df.duplicated().sum()
df.isnull().sum()

In [ ]:
# Renomear as colunas em pt-br pra facilitar a vida
df = df.rename(columns={
    'region': 'regiao',
    'category': 'categoria',
    'parameter': 'parametro',
    'mode': 'tipo_veiculo',
    'powertrain': 'motorizacao',
    'year': 'ano',
    'unit': 'unidade',
    'value': 'valor'
})

FILTROS 

In [ ]:
# Filtrar somente vendas anuais (parametro = 'EV sales' e unidade = 'Vehicles')
df_vendas = df[(df['parametro'] == 'EV sales') & (df['unidade'] == 'Vehicles')]

# Visualizar as primeiras linhas para confirmar
print(df_vendas.head())
print(df_vendas.columns)


In [ ]:
# Transformar a coluna valor (quantidade de veículos vendidos) em uma variável categórica, dividindo as vendas em faixas: “baixo”, “médio” e “alto” para classificação.
# Usamos a função pd.qcut() para criar as três faixas baseadas nos tercis (ou seja, dividindo os dados em três partes iguais).
# Criando a variável-alvo: faixa de vendas
df_vendas['faixa_vendas'] = pd.qcut(df_vendas['valor'], q=3, labels=['baixo', 'medio', 'alto'])

# Conferindo o resultado
print(df_vendas[['regiao', 'ano', 'valor', 'faixa_vendas']].sample(10))
print(df_vendas['faixa_vendas'].value_counts())


PRÉ PROCESSAMENTO

In [ ]:
#  Separação das variáveis
# Features e variável-alvo
features = ['regiao', 'ano', 'motorizacao']  # Ajuste conforme desejar
X = df_vendas[features]
y = df_vendas['faixa_vendas']


In [ ]:
# Separar conjunto de treino e teste

# Separar os dados
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)


In [ ]:
# Transformar variáveis categóricas
# Como temos categorias (ex: países, tipos de motorização), precisamos codificá-las.
# O mais comum é usar OneHotEncoder.

# Usar o argumento correto para a versão mais recente do scikit-learn
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Ajustar e transformar os dados
X_train_enc = ohe.fit_transform(X_train)
X_test_enc = ohe.transform(X_test)


In [ ]:
# Conferência
print("Shape do X_train_enc:", X_train_enc.shape)
print("Shape do X_test_enc:", X_test_enc.shape)
print("Classes alvo:", y.unique())


ARVORE DE DECISÃO

In [ ]:
#  Treinando o Classificador (Árvore de Decisão)
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Treinamento do modelo
dt = DecisionTreeClassifier(max_depth=5, random_state=42)
dt.fit(X_train_enc, y_train)

# Previsões
y_pred = dt.predict(X_test_enc)


In [ ]:
# Avaliando o Modelo
# Relatório de classificação
print(classification_report(y_test, y_pred))

# Matriz de confusão
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred))


In [ ]:
# Visualização da Árvore de Decisão

from sklearn import tree
import matplotlib.pyplot as plt

plt.figure(figsize=(22,10))
tree.plot_tree(
    dt,
    feature_names=ohe.get_feature_names_out(),
    class_names=dt.classes_,
    filled=True,
    fontsize=9,
    rounded=True
)
plt.show()


RANDOM FOREST

In [ ]:
# Treinando o Random Forest
from sklearn.ensemble import RandomForestClassifier

# Treinamento do modelo Random Forest
rf = RandomForestClassifier(n_estimators=100, max_depth=7, random_state=42)
rf.fit(X_train_enc, y_train)

# Previsões
y_pred_rf = rf.predict(X_test_enc)


In [ ]:
# Avaliando o Modelo Random Forest

from sklearn.metrics import classification_report, confusion_matrix

print("=== Random Forest ===")
print(classification_report(y_test, y_pred_rf))
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred_rf))


In [ ]:
# Importância das Variáveis

import numpy as np

importances = rf.feature_importances_
feature_names = ohe.get_feature_names_out()
indices = np.argsort(importances)[::-1]

print("Importância das variáveis (top 10):")
for i in range(min(10, len(feature_names))):
    print(f"{feature_names[indices[i]]}: {importances[indices[i]]:.4f}")


COMPARATIVO 

In [33]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Função para gerar um dicionário de métricas
def get_metrics(y_true, y_pred, modelo):
    return {
        'Modelo': modelo,
        'Acurácia': accuracy_score(y_true, y_pred),
        'Precisão (macro)': precision_score(y_true, y_pred, average='macro'),
        'Recall (macro)': recall_score(y_true, y_pred, average='macro'),
        'F1-score (macro)': f1_score(y_true, y_pred, average='macro')
    }

# Métricas da Árvore de Decisão
metrics_dt = get_metrics(y_test, y_pred, 'Árvore de Decisão')

# Métricas do Random Forest
metrics_rf = get_metrics(y_test, y_pred_rf, 'Random Forest')

# Organizar em DataFrame para visualização comparativa
import pandas as pd
df_comparacao = pd.DataFrame([metrics_dt, metrics_rf])

print(df_comparacao)


              Modelo  Acurácia  Precisão (macro)  Recall (macro)  \
0  Árvore de Decisão  0.486352          0.643853        0.486807   
1      Random Forest  0.602978          0.645902        0.603409   

   F1-score (macro)  
0          0.467892  
1          0.606052  
